# Chap 1. 신경망 복습

##### 1.1. 수학과 파이썬 복습은 패스하겠습니다

## 1.2. 신경망의 추론


### 신경망
- 입력을 출력으로 변환하는 단순한 함수
- 입력층, 은닉층, 출력층으로 구성
- 가중치를 학습시켜 원하는 형태 구성 및 이전 뉴런의 값에 영향을 받지 않는 편향 존재

![](img/chap_01/1.png)

- 은닉층의 첫번째 뉴런의 계산 공식
    - ![](img/chap_01/2.png)

- 실제 완전연결 계층이 수행하는 변환을 다음과 같이 행렬의 곱으로 정리 가능
    - ![](img/chap_01/3.png)
    - 다음과 같이 간소화 할 수 있음
        - ![](img/chap_01/4.png)
        
- 샘플 데이터에 따른 데이터 형상 확인

![](img/chap_01/5.png)
![](img/chap_01/6.png)

### 선형 변환과 비선형 변환
- 일반적인 완전연결계층에 의한 변환은 선형 변환임
- 선형 변환만으로는 은닉층이 없는 네트워크와 다를 바가 없음
- 비선형 활성화 함수를 이용하여 신경망의 표현력을 높일 수 있음

#### Sigmoid function
- 기본적인 활성화 함수
- 임의의 실수를 입력받아 0 ~ 1 사이의 실수를 출력
- 선형 회귀에서 많이 사용

![](img/chap_01/7.png)

![](img/chap_01/8.png)

### Affine 계층과 Sigmoid 계층의 순전파
- 각 계층을 파이썬 클래스로 구현
- 모든 계층은 forward()와 backward()메서드를 가짐
- 모든 계층은 params와 grads를 인스턴스 변수로 가짐
- params: 가중치 및 편향 등의 값을 담는 리스트
- grads: params에 저장된 매개변수에 대응하는 기울기를 담는 리스트

## 1.3. 신경망의 학습


### Loss function
- 신경망의 학습이 얼마나 잘 되고 있는가를 알기 위한 척도 필요
- 손실을 사용하여 학습 단계의 특정 시점의 신경망의 성능을 나타냄
- 손실은 정답 라벨과 신경망이 예측한 결과를 비교하여 예측의 차이를 나타내는 스칼라 값

### Softmax
- n개의 입력에 대해 n개의 출력을 내보내며, 각 출력은 0.0 이상 1.0 이하의 실수, 모든 출력의 합은 1.0 => 각 출력을 확률로 해석 가능
- 출력이 총 n개일 때, k번째 출력 y_k를 구하는 계산식
    
    ![](img/chap_01/10.png)

### Cross Entropy Error
- 다중 클래스 분류 신경망에서 흔히 사용하는 손실 함수
- 신경망이 출력하는 각 클래스의 확률과 정답 레이블을 이용해 구할 수 있음
- 출력이 총 k개일 때, cross entropy error를 구하는 계산식

    ![](img/chap_01/11.png)
    - t_k는 k번째 클래스에 해당하는 정답 레이블(0 or 1)
    - 정답 레이블은 t = [0, 0, 1]과 같이 ont-hot vector로 표기
    - y_k는 k번째 출력의 softmax를 계산한 확률값
    
- 실제로 cross entropy error은 정답 레이블이 1인 원소의 출력값의 자연로그 값
- 다른 원소들은 t_k(정답레이블)가 0이기 때문에 결과에 영향을 주지 못함

#### 미니 배치를 고려한 Cross Entropy Error
- 데이터가 N개이며, t_nk는 n번째 데이터의 k차원째의 값(0 or 1)
- 데이터를 N개로 확장하였고, 이를 N으로 나눠 1개당 평균 손실함수를 구함
    
    ![](img/chap_01/12.png)

![](img/chap_01/9.png)
       
#### 일반적으로 softmax와 cross entropy를 Softmax with Loss 계층 하나로 구현하며, 이로인해 역전파 계산이 쉬워짐
![](img/chap_01/13.png) 

### 미분과 기울기
- 벡터의 각 원소에 대한 미분을 정리한 것이 기울기

    ![](img/chap_01/15.png) 

- 벡터와 마찬가지로 행렬에서도 기울기를 정의할 수 있음
    - W가 m x n 행렬이라면, L = g(W) 함수의 기울기는 다음과 같음
    - 이때, W와 $\frac{\partial L}{\partial W}$의 형상이 같음
    
        ![](img/chap_01/14.png) 
        
        
### 오차역전파
- 학습 시 신경망은 학습 데이터를 입력하면 손실을 출력
- 각 매개변수에 대한 손실의 기울기를 구하고 싶음
- 오차역전파를 사용하여 손쉽게 기울기를 구할 수 있음

### 연쇄법칙
- 합성함수에 대한 미분 법칙
- 아무리 많은 함수를 연결해도, 각 함수의 국소적인 미분을 계산할 수 있다면 그 값들을 곱해서 전체의 미분을 구할 수 있음

### 각 노드별 순전파 및 역전파 개념 및 그래프
#### 덧셈 노드
- $\frac{\partial z}{\partial x}$와 $\frac{\partial z}{\partial y}$ 모두 1
- 덧셈 노드는 상류로부터 받은 값에 1을 곱하여 하류로 기울기를 전파 = 기울기를 그대로 전파

![](img/chap_01/16.png) 

#### 곱셈 노드
- $\frac{\partial z}{\partial x} = y$, $\frac{\partial z}{\partial y} = x$
- 곱셈 노드는 순전파 시의 입력을 서로 바꾼 값을 곱함

![](img/chap_01/17.png) 

#### 분기 노드
- 같은 값이 복제되어 분기되었기 때문에 역전파는 상류에서 온 기울기들의 합

![](img/chap_01/18.png)

#### Repeat 노드
- 분기노드를 일반화한 노드

![](img/chap_01/19.png) 

#### Sum 노드
- 범용 덧셈 노드로, $N \times D$ 배열을 0축(axis=0)에 대한 합을 구함
- Sum 노드의 역전파는 상류로부터의 기울기를 하류로 분배 => 덧셈 노드의 확장
- Sum 노드와 Repeat 노드는 서로 반대 관계

![](img/chap_01/20.png) 

#### MATtatMul
 ㄴ노녿노드

## 1.4. 신경망으로 문제를 풀다


##### 1.5. 계산 고속화는 패스하겠습니다.

# Chap 2. 자연어와 단어의 분산 표현

## 2.1. 자연어 처리란
- 우리가 사용하는 말을 컴퓨터에게 이해시키기 위한 기술
- 컴퓨터에게 자연어를 이해시킬 수 있다면, 검색엔진, 기계번역 등과 같은 기술에 널리 사용 가능

### 단어의 의미
- 말은 문자로 구성되며, 말의 의미는 단어로 구성
- 단어란, 의미의 최소단위이며, 컴퓨터에게 단어의 의미를 이해시키는 것은 중요
- '시소러스', '통계 기반 기법', '추론 기반 기법' 등을 통해 단어의 의미를 표현할 수 있음

## 2.2. 시소러스
- 우리가 각각의 단어에 의미를 설명해 넣는 것처럼, 컴퓨터가 이해할 수 있는 형태의 사전 형식
- 뜻이 같은 단어나 뜻이 비슷한 단어가 한 그룹으로 분류

    ![](img/chap_01/21.png) 
    
- 단어 사이의 상위-하위 혹은 전체-부분 등 의 관계까지 묘사할 수 있음

    ![](img/chap_01/21.png) 
    
- 이러한 단어 네트워크를 통해 컴퓨가 간접적으로 단어의 의미를 이해했다고 할 수 있음
- 자연어 처리 분야에서 WordNet이라는 시소러스가 가장 유명함

### 시소러스의 문제점
- 시대 변화에 대응하기 어려움
- 사람을 쓰는 비용이 큼
- 단어의 미묘한 차이를 표현할 수 없음

#### => 이를 해결하기 위해 통계 기반 기법과 신경망을 사용한 추론 기반 기법을 이용

## 2.3. 통계 기반 기법


## 2.4. 통계 기반 기법 개선하기